In [2]:
import warnings
import pandas as pd
import pyodbc
import numpy as np

warnings.simplefilter('ignore')

# Connection details for the source SQL Server database
source_db = {
    'servername': r'LAPTOP-NDOACQTH\SQLEXPRESS',
    'database': 'AdventureWorks2019'
}

source_db_northwind = {
    'servername': r'LAPTOP-NDOACQTH\SQLEXPRESS',
    'database': 'NORTHWIND'
}

# Connection details for the destination SQL Server database
destination_db = {
    'servername': r'LAPTOP-NDOACQTH\SQLEXPRESS',
    'database': 'DWUnitedOutdoors'
}

# Connection string for the source SQL Server database
source_conn_str = (
    'Driver={SQL Server};'
    'SERVER=' + source_db['servername'] + ';'
    'DATABASE=' + source_db['database'] + ';'
    'Trusted_Connection=yes;'
)

# Connection string for the source SQL Server database
source_conn_str_northwind = (
    'Driver={SQL Server};'
    'SERVER=' + source_db_northwind['servername'] + ';'
    'DATABASE=' + source_db_northwind['database'] + ';'
    'Trusted_Connection=yes;'
)

# Connection string for the destination SQL Server database
destination_conn_str = (
    'Driver={SQL Server};'
    'SERVER=' + destination_db['servername'] + ';'
    'DATABASE=' + destination_db['database'] + ';'
    'Trusted_Connection=yes;'
)

In [3]:
#Person.Person
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        person_df = pd.read_sql_query("SELECT * FROM Person.person", source_conn)
        personPhone_df = pd.read_sql_query("SELECT PhoneNumber, PhoneNumberTypeID, BusinessEntityID FROM Person.PersonPhone", source_conn)
        phoneNumberType_df = pd.read_sql_query("SELECT PhoneNumberTypeID, Name FROM Person.PhoneNumberType", source_conn)
        personEmailAdress_df = pd.read_sql_query("SELECT EmailAddressID, EmailAddress, BusinessEntityID FROM Person.EmailAddress", source_conn)
        personPassword_df = pd.read_sql_query("SELECT PasswordHash, PasswordSalt, BusinessEntityID FROM Person.Password", source_conn)

        # Merging DataFrames
        person_df = pd.merge(person_df, personPhone_df, on='BusinessEntityID', how='left')
        person_df = pd.merge(person_df, phoneNumberType_df, on='PhoneNumberTypeID', how='left')
        person_df = pd.merge(person_df, personEmailAdress_df, on='BusinessEntityID', how='left')
        person_df = pd.merge(person_df, personPassword_df, on='BusinessEntityID', how='left')

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Person]
        (BusinessEntityID, PersonType, NameStyle, Title, FirstName, MiddleName, LastName, Suffix, EmailPromotion, PasswordSalt, PasswordHash, EmailAddress, Name, EmailAddressID, PhoneNumberTypeID)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = person_df[['BusinessEntityID', 'PersonType', 'NameStyle', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix', 'EmailPromotion', 'PasswordSalt', 'PasswordHash', 'EmailAddress', 'Name', 'EmailAddressID', 'PhoneNumberTypeID']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [4]:
#businessEntity_df
# Establish connection to the source SQL Server database and read
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        businessEntity_df = pd.read_sql_query("SELECT BusinessEntityID FROM Person.BusinessEntity", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[BusinessEntity]
        (BusinessEntityID)
        VALUES (?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = businessEntity_df[['BusinessEntityID']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)
# businessEntity_df

Data inserted successfully.


In [5]:
#businessEntityAddress_df
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        businessEntityAddress_df = pd.read_sql_query("SELECT AddressID, AddressTypeID, BusinessEntityID FROM Person.BusinessEntityAddress", source_conn)
        addressType_df = pd.read_sql_query("SELECT AddressTypeID, Name FROM Person.AddressType", source_conn)
                                           
        # Merging DataFrames
        businessEntityAddress_df = pd.merge(businessEntityAddress_df, addressType_df, on='AddressTypeID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[BusinessEntityAddress]
        (BusinessEntityID, AddressID, AddressTypeID, Name)
        VALUES (?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = businessEntityAddress_df[['BusinessEntityID', 'AddressID', 'AddressTypeID', 'Name']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [6]:
#person.Address_df
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        personAddress_df = pd.read_sql_query("SELECT AddressID, AddressLine1, AddressLine2, City, StateProvinceID, PostalCode, SpatialLocation FROM Person.Address", source_conn)
                                           
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Address]
        (AddressID, AddressLine1, AddressLine2, City, StateProvinceID, PostalCode, SpatialLocation)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = personAddress_df[['AddressID', 'AddressLine1', 'AddressLine2', 'City', 'StateProvinceID', 'PostalCode', 'SpatialLocation']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [7]:
#Sales.CreditCard
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        creditCard_df = pd.read_sql_query("SELECT CreditCardID, CardType, CardNumber, ExpMonth, ExpYear FROM Sales.CreditCard", source_conn)
        personCreditCard_df = pd.read_sql_query("SELECT CreditCardID, BusinessEntityID FROM Sales.PersonCreditCard", source_conn)
        
        # Merging DataFrames
        creditCard_df = pd.merge(creditCard_df, personCreditCard_df, on='CreditCardID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

#Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[CreditCard]
        (CreditCardID, BusinessEntityID, CardType, CardNumber, ExpMonth, ExpYear)
        VALUES (?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = creditCard_df[['CreditCardID', 'BusinessEntityID', 'CardType', 'CardNumber', 'ExpMonth', 'ExpYear']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [8]:
#Sales.SalesPerson
# Establish connection to the source SQL Server database and read data  
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        salesPerson_df = pd.read_sql_query("SELECT BusinessEntityID, TerritoryID, SalesQuota, Bonus, CommissionPct, SalesLastYear, SalesYTD FROM Sales.SalesPerson", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Fill NaN values in selected columns
salesPerson_df['TerritoryID'].fillna(0.0, inplace=True)
salesPerson_df['SalesQuota'].fillna(0.0, inplace=True)
salesPerson_df['CommissionPct'].fillna(0.0, inplace=True)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
         
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[SalesPerson]
        (BusinessEntityID, TerritoryID, SalesQuota, Bonus, CommisionPct, SalesLastYear, SalesYTD)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """
         
        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = salesPerson_df[['BusinessEntityID', 'TerritoryID', 'SalesQuota', 'Bonus', 'CommissionPct', 'SalesLastYear', 'SalesYTD']].values.tolist()
         
        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)
         
        # Commit transaction
        destination_conn.commit()
         
        print("Data inserted successfully into SalesPerson table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into SalesPerson table.


In [9]:
#Sales.Store
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        salesStore_df = pd.read_sql_query("SELECT BusinessEntityID, Name, SalesPersonID FROM Sales.Store", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for salesPersonQuotaHistory_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Store]
        (BusinessEntityID, Name, SalesPersonID)
        VALUES (?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = salesStore_df[['BusinessEntityID', 'Name', 'SalesPersonID']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into salesstore table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into salesstore table.


In [10]:
#Sales.SalesPersonQuotaHistory
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        salesPersonQuotaHistory_df = pd.read_sql_query("SELECT BusinessEntityID, QuotaDate, SalesQuota FROM Sales.SalesPersonQuotaHistory", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for salesPersonQuotaHistory_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[SalesPersonQuotaHistory]
        (BusinessEntityID, QuotaDate, SalesQuota)
        VALUES (?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = salesPersonQuotaHistory_df[['BusinessEntityID', 'QuotaDate', 'SalesQuota']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into SalesPersonQuotaHistory table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into SalesPersonQuotaHistory table.


In [11]:
#Sales.SalesTerritoryHistory
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        salesTerritoryHistory_df = pd.read_sql_query("SELECT BusinessEntityID, TerritoryID, StartDate, EndDate FROM Sales.salesTerritoryHistory", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Convert date columns to datetime objects
date_columns = ['StartDate', 'EndDate']
for column in date_columns:
    salesTerritoryHistory_df[column] = pd.to_datetime(salesTerritoryHistory_df[column], errors='coerce')

# Drop rows with invalid dates
salesTerritoryHistory_df.dropna(subset=date_columns, inplace=True)

# Format dates as strings in 'YYYY-MM-DD' format
salesTerritoryHistory_df['StartDate'] = salesTerritoryHistory_df['StartDate'].dt.strftime('%Y-%m-%d')
salesTerritoryHistory_df['EndDate'] = salesTerritoryHistory_df['EndDate'].dt.strftime('%Y-%m-%d')

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for salesTerritoryHistory_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[SalesTerritoryHistory]
        (BusinessEntityID, TerritoryID, StartDate, EndDate)
        VALUES (?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = salesTerritoryHistory_df[['BusinessEntityID', 'TerritoryID', 'StartDate', 'EndDate']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into SalesTerritoryHistory table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into SalesTerritoryHistory table.


In [12]:
#Sales.SalesTerritory
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        SalesTerritory_df = pd.read_sql_query("SELECT TerritoryID, Name, CountryRegionCode, [Group], SalesYTD, SalesLastYear, CostYTD, CostLastYear FROM Sales.SalesTerritory", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for SalesTerritory_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[SalesTerritory]
        (TerritoryID, Name, CountryRegionCode, [Group], SalesYTD, SalesLastYear, CostYTD, CostLastYear)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = SalesTerritory_df[['TerritoryID', 'Name', 'CountryRegionCode', 'Group', 'SalesYTD', 'SalesLastYear', 'CostYTD', 'CostLastYear']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into SalesTerritory table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into SalesTerritory table.


In [13]:
#Sales.CountryRegionCurrency
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        CountryRegionCurrency_df = pd.read_sql_query("SELECT CountryRegionCode, CurrencyCode FROM Sales.CountryRegionCurrency",source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for CountryRegionCurrency_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[CountryRegionCurrency]
        (CountryRegionCode, CurrencyCode)
        VALUES (?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = CountryRegionCurrency_df[['CountryRegionCode', 'CurrencyCode']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into CountryRegionCurrency table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into CountryRegionCurrency table.


In [14]:
#Sales.Currency
# Establish connection to the source SQL Server database and read data
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        currency_df = pd.read_sql_query("SELECT CurrencyCode, Name FROM Sales.Currency",source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for currency_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Currency]
        (CurrencyCode, Name)
        VALUES (?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = currency_df[['CurrencyCode', 'Name']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into Currency table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into Currency table.


In [102]:
#Sales.order
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        sales_order_header_df = pd.read_sql_query("SELECT SalesOrderID, OrderDate, DueDate, ShipDate, Status, SalesOrderNumber, PurchaseOrderNumber, AccountNumber, CustomerID, SalesPersonID, TerritoryID, BillToAddressID, ShipToAddressID, ShipMethodID, CurrencyRateID, CreditCardID, SubTotal, TaxAmt, Freight   FROM Sales.SalesOrderHeader", source_conn)
        sales_order_detail_df = pd.read_sql_query("SELECT SalesOrderDetailID, CarrierTrackingNumber , OrderQty, ProductID,SpecialOfferID,UnitPrice, UnitPriceDiscount, LineTotal, SalesOrderID  FROM Sales.SalesOrderDetail ", source_conn)
        currency_rate_df = pd.read_sql_query("SELECT CurrencyRateID, CurrencyRateDate,FromCurrencyCode, ToCurrencyCode, AverageRate, EndOfDayRate  FROM Sales.CurrencyRate", source_conn)
        
        ## merging dataframes
        sales_order_df = pd.merge(sales_order_header_df, currency_rate_df, on='CurrencyRateID', how='left')
        sales_order_df = pd.merge(sales_order_detail_df, sales_order_df, on='SalesOrderID', how='left')
                           
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
    
sales_order_df['OrderDate'] = pd.to_datetime(sales_order_df['OrderDate'], errors='coerce')
sales_order_df['DueDate'] = pd.to_datetime(sales_order_df['DueDate'], errors='coerce')
sales_order_df['ShipDate'] = pd.to_datetime(sales_order_df['ShipDate'], errors='coerce')
sales_order_df['CurrencyRateDate'] = pd.to_datetime(sales_order_df['CurrencyRateDate'], errors='coerce')

# Replace NaT values with a default datetime value
default_date = pd.Timestamp('1900-01-01')  # or any other default value you prefer
sales_order_df['OrderDate'].fillna(default_date, inplace=True)
sales_order_df['DueDate'].fillna(default_date, inplace=True)
sales_order_df['ShipDate'].fillna(default_date, inplace=True)
sales_order_df['CurrencyRateDate'].fillna(default_date, inplace=True)
sales_order_df['Freight'].fillna(0.0, inplace=True)
sales_order_df['FromCurrencyCode'].fillna('default', inplace=True)
sales_order_df['ToCurrencyCode'].fillna('default', inplace=True)
sales_order_df['AverageRate'].fillna(0.0, inplace=True)
sales_order_df['EndOfDayRate'].fillna(0.0, inplace=True)
sales_order_df['CurrencyRateID'].fillna(0, inplace=True)
sales_order_df['Status'].fillna(0, inplace=True)
sales_order_df['CarrierTrackingNumber'].fillna('Default', inplace=True)
sales_order_df['SalesPersonID'].fillna(0, inplace=True)
sales_order_df['TaxAmt'].fillna(0.0, inplace=True)
sales_order_df['CreditCardID'].fillna(0, inplace=True)

sales_order_df['CreditCardID'] = sales_order_df['CreditCardID'].astype(int)
sales_order_df['CurrencyRateID'] = sales_order_df['CurrencyRateID'].astype(int)
sales_order_df['SalesPersonID'] = sales_order_df['SalesPersonID'].astype(int)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for sales_order_df
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Sales_Order]
        (SalesOrderID, OrderDate, DueDate, ShipDate, Status, SalesOrderNumber, PurchaseOrderNumber, AccountNumber, CustomerID, SalesPersonID, TerritoryID, BillToAdressID, ShipToAdressID, ShipMethodID, CurrencyRateID, CreditCardID, SubTotal, TaxAmt, Freight, SalesOrderDetailID, CarrierTrackingNumber, OrderQty, ProductID, UnitPrice, UnitPriceDiscount, LineTotal, CurrencyRateDate, FromCurrencyCode, ToCurrencyCode, AverageRate, EndOfDayRate)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = sales_order_df[['SalesOrderID', 'OrderDate', 'DueDate', 'ShipDate', 'Status', 'SalesOrderNumber', 'PurchaseOrderNumber', 'AccountNumber', 'CustomerID', 'SalesPersonID', 'TerritoryID', 'BillToAddressID', 'ShipToAddressID', 'ShipMethodID', 'CurrencyRateID', 'CreditCardID', 'SubTotal', 'TaxAmt', 'Freight', 'SalesOrderDetailID', 'CarrierTrackingNumber', 'OrderQty', 'ProductID', 'UnitPrice', 'UnitPriceDiscount', 'LineTotal', 'CurrencyRateDate', 'FromCurrencyCode', 'ToCurrencyCode', 'AverageRate', 'EndOfDayRate']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into SalesOrder table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into SalesOrder table.


,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,SalesOrderID,OrderDate,...,CurrencyRateID,CreditCardID,SubTotal,TaxAmt,Freight,CurrencyRateDate,FromCurrencyCode,ToCurrencyCode,AverageRate,EndOfDayRate
0,1,4911-403C-98,1,776,1,2024.994,0.0,2024.994,43659,2011-05-31,...,0,16281,20565.6206,1971.5149,616.0984,1900-01-01,default,default,0.0,0.0
1,2,4911-403C-98,3,777,1,2024.994,0.0,6074.982,43659,2011-05-31,...,0,16281,20565.6206,1971.5149,616.0984,1900-01-01,default,default,0.0,0.0
2,3,4911-403C-98,1,778,1,2024.994,0.0,2024.994,43659,2011-05-31,...,0,16281,20565.6206,1971.5149,616.0984,1900-01-01,default,default,0.0,0.0
3,4,4911-403C-98,1,771,1,2039.994,0.0,2039.994,43659,2011-05-31,...,0,16281,20565.6206,1971.5149,616.0984,1900-01-01,default,default,0.0,0.0
4,5,4911-403C-98,1,772,1,2039.994,0.0,2039.994,43659,2011-05-31,...,0,16281,20565.6206,1971.5149,616.0984,1900-01-01,default,default,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121312,121313,Default,1,878,1,21.980,0.0,21.980,75122,2014-06-30,...,0,18719,30.9700,2.4776,0.7743,1900-01-01,default,default,0.0,0.0
121313,121314,Default,1,712,1,8.990,0.0,8.990,75122,2014-06-30,...,0,18719,30.9700,2.4776,0.7743,1900-01-01,default,default,0.0,0.0
121314,121315,Default,1,878,1,21.980,0.0,21.980,75123,2014-06-30,...,0,10084,189.9700,15.1976,4.7493,1900-01-01,default,default,0.0,0.0
121315,121316,Default,1,879,1,159.000,0.0,159.000,75123,2014-06-30,...,0,10084,189.9700,15.1976,4.7493,1900-01-01,default,default,0.0,0.0


In [16]:
#purchasing.shipmethod
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        ship_method_df = pd.read_sql_query("SELECT ShipMethodID, Name, ShipBase, ShipRate FROM Purchasing.ShipMethod", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for ship_method
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ShipMethod]
        (ShipMethodID, Name, ShipBase, ShipRate)
        VALUES (?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = ship_method_df[['ShipMethodID', 'Name', 'ShipBase', 'ShipRate']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into ShipMethod table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into ShipMethod table.


In [17]:
#Sales.salesOrderHeaderSalesReason
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        salesOrderHeaderSalesReason_df = pd.read_sql_query("SELECT SalesOrderID, SalesReasonID FROM Sales.SalesOrderHeaderSalesReason", source_conn)
        salesReason_df = pd.read_sql_query("SELECT Name, ReasonType, SalesReasonID FROM Sales.SalesReason", source_conn)

        ## merging dataframes
        salesOrderHeaderSalesReason_df = pd.merge(salesOrderHeaderSalesReason_df, salesReason_df, on='SalesReasonID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for salesOrderHeaderSalesReason
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[salesOrderHeaderSalesReason]
        (SalesOrderID, SalesReasonID, Name, ReasonType)
        VALUES (?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = salesOrderHeaderSalesReason_df[['SalesOrderID', 'SalesReasonID', 'Name', 'ReasonType']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into salesOrderHeaderSalesReason table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into salesOrderHeaderSalesReason table.


In [18]:
#Person.CountryRegion
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        CountryRegion_df = pd.read_sql_query("SELECT CountryRegionCode, Name FROM Person.CountryRegion", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for CountryRegion
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[CounrtyRegion]
        (CountryRegionCode, Name)
        VALUES (?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = CountryRegion_df[['CountryRegionCode', 'Name']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into CountryRegion table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into CountryRegion table.


In [19]:
#Person.StateProvince
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        StateProvince_df = pd.read_sql_query("SELECT StateProvinceID, StateProvinceCode, CountryRegionCode, Name, TerritoryID FROM Person.StateProvince", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for StateProvince
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[StateProvince]
        (StateProvinceID, StateProvinceCode, CountryRegionCode, Name, TerritoryID)
        VALUES (?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = StateProvince_df[['StateProvinceID', 'StateProvinceCode', 'CountryRegionCode', 'Name', 'TerritoryID']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into StateProvince table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into StateProvince table.


In [20]:
#Person.BusinessEntityContact
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        BusinessEntityContact_df = pd.read_sql_query("SELECT BusinessEntityID, PersonID, ContactTypeID FROM Person.BusinessEntityContact", source_conn)
        ContactType_df = pd.read_sql_query("SELECT Name, ContactTypeID FROM Person.ContactType", source_conn)
        
        ## merging dataframes
        BusinessEntityContact_df = pd.merge(BusinessEntityContact_df, ContactType_df, on='ContactTypeID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for BusinessEntityContact
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[BusinessEntityContact]
        (BusinessEntityID, PersonID, ContactTypeID, Name)
        VALUES (?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = BusinessEntityContact_df[['BusinessEntityID', 'PersonID', 'ContactTypeID', 'Name']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into BusinessEntityContact table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into BusinessEntityContact table.


In [22]:
#Purchasing.PurchaseOrder
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        PurchaseOrderDetail_df = pd.read_sql_query("SELECT PurchaseOrderID, PurchaseOrderDetailID, DueDate, OrderQty, ProductID, UnitPrice, LineTotal, ReceivedQty, RejectedQty, StockedQty FROM Purchasing.PurchaseOrderDetail", source_conn)
        PurchaseOrderHeader_df = pd.read_sql_query("SELECT PurchaseOrderID, RevisionNumber, Status, EmployeeID, VendorID, ShipMethodID, OrderDate, ShipDate, Subtotal, taxAmt, Freight, TotalDue FROM Purchasing.PurchaseOrderHeader ", source_conn)
        
        ## merging dataframes
        PurchaseOrderDetail_df = pd.merge(PurchaseOrderHeader_df, PurchaseOrderDetail_df, on='PurchaseOrderID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[PurchaseOrder]
        (PurchaseOrderID, PurchaseOrderDetailID, DueDate, OrderQty, ProductID, UnitPrice, LineTotal, ReceivedQty, StockedQuantity, RejectedQty, RevisionNumber, Status, EmployeeID, VendorID, ShipMethodID, OrderDate, ShipDate, Subtotal, taxAmt, TotalDue)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? )
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = PurchaseOrderDetail_df[['PurchaseOrderID', 'PurchaseOrderDetailID', 'DueDate', 'OrderQty', 'ProductID', 'UnitPrice', 'LineTotal', 'ReceivedQty', 'StockedQty', 'RejectedQty', 'RevisionNumber', 'Status', 'EmployeeID', 'VendorID', 'ShipMethodID', 'OrderDate', 'ShipDate', 'Subtotal', 'taxAmt', 'TotalDue' ]].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

Data inserted successfully.


In [23]:
#Purchasing.Vendor
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        Vendor_df = pd.read_sql_query("SELECT BusinessEntityID, AccountNumber, Name, CreditRating, PurchasingWebServiceURL FROM Purchasing.Vendor", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Vendor]
        (BusinessEntityID, AccountNumber, Name, CreditRating, PurchasingWebServiceURL)
        VALUES (?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = Vendor_df[['BusinessEntityID', 'AccountNumber', 'Name', 'CreditRating', 'PurchasingWebServiceURL']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()
        
        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully.


In [24]:
##Production.ProductSubCategory
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        ProductSubCategory_df = pd.read_sql_query("SELECT ProductSubCategoryID ,ProductCategoryID, Name FROM Production.ProductSubCategory", source_conn)
        ProductCategory_df = pd.read_sql_query("SELECT Name, ProductCategoryID FROM Production.ProductCategory", source_conn)

        ## merging dataframes
        ProductSubCategory_df = pd.merge(ProductSubCategory_df, ProductCategory_df, on='ProductCategoryID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
    # Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ProductSubCategory]
        (ProductSubCategoryID, ProductCategoryID, SubCategory_Name, Category_Name)
        VALUES (?, ?, ?, ?)
        """
        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = ProductSubCategory_df[['ProductSubCategoryID', 'ProductCategoryID', 'Name_x', 'Name_y']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [128]:
#Production.Product
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        productionproduct_df = pd.read_sql_query("SELECT ProductID ,Name, ProductNumber, Color, SafetyStockLevel, ReorderPoint, StandardCost, ListPrice, Size, SizeUnitMeasureCode, WeightUnitMeasureCode, Weight, DaysToManufacture, ProductLine, Class, Style, ProductSubcategoryID, SellStartDate, SellEndDate, DiscontinuedDate FROM Production.Product", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
    
# Convert date fields to datetime objects
productionproduct_df['SellStartDate'] = pd.to_datetime(productionproduct_df['SellStartDate'])
productionproduct_df['SellEndDate'] = pd.to_datetime(productionproduct_df['SellEndDate'])
productionproduct_df['DiscontinuedDate'] = pd.to_datetime(productionproduct_df['DiscontinuedDate'])

# Convert datetime objects to string format
productionproduct_df['SellStartDate'] = productionproduct_df['SellStartDate'].dt.strftime('%Y-%m-%d %H:%M:%S')
productionproduct_df['SellEndDate'].fillna('1900-01-01 00:00:00', inplace=True)
productionproduct_df['DiscontinuedDate'].fillna('1900-01-01 00:00:00', inplace=True)

# Fill missing values with defaults
productionproduct_df['Color'].fillna('Unknown', inplace=True)
productionproduct_df['Size'].fillna('null', inplace=True)
productionproduct_df['SizeUnitMeasureCode'].fillna('nul', inplace=True)
productionproduct_df['Weight'].fillna(0.0, inplace=True)
productionproduct_df['ProductLine'].fillna('no', inplace=True)
productionproduct_df['Class'].fillna('no', inplace=True)
productionproduct_df['Style'].fillna('no', inplace=True)
productionproduct_df['ProductSubcategoryID'].fillna(0, inplace=True)


# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ProductionProduct]
        (ProductID, Name, ProductNumber, Color, SafetyStockLevel, ReorderPoint, StandardCost, ListPrice, Size, SizeUnitMeasureCode, WeightUnitMeasureCode, Weight, DaysToManufacture, ProductLine, Class, Style, ProductSubcategoryID, SellStartDate, SellEndDate, DiscontinuedDate)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = productionproduct_df[['ProductID', 'Name', 'ProductNumber', 'Color', 'SafetyStockLevel', 'ReorderPoint', 'StandardCost', 'ListPrice', 'Size', 'SizeUnitMeasureCode', 'WeightUnitMeasureCode', 'Weight', 'DaysToManufacture', 'ProductLine', 'Class', 'Style', 'ProductSubcategoryID', 'SellStartDate', 'SellEndDate', 'DiscontinuedDate']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


,ProductID,Name,ProductNumber,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,SellStartDate,SellEndDate,DiscontinuedDate
0,1,Adjustable Race,AR-5381,Unknown,1000,750,0.0000,0.00,null,nul,None,0.00,0,no,no,no,0.0,2008-04-30 00:00:00,1900-01-01,1900-01-01
1,2,Bearing Ball,BA-8327,Unknown,1000,750,0.0000,0.00,null,nul,None,0.00,0,no,no,no,0.0,2008-04-30 00:00:00,1900-01-01,1900-01-01
2,3,BB Ball Bearing,BE-2349,Unknown,800,600,0.0000,0.00,null,nul,None,0.00,1,no,no,no,0.0,2008-04-30 00:00:00,1900-01-01,1900-01-01
3,4,Headset Ball Bearings,BE-2908,Unknown,800,600,0.0000,0.00,null,nul,None,0.00,0,no,no,no,0.0,2008-04-30 00:00:00,1900-01-01,1900-01-01
4,316,Blade,BL-2036,Unknown,800,600,0.0000,0.00,null,nul,None,0.00,1,no,no,no,0.0,2008-04-30 00:00:00,1900-01-01,1900-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,BB-8107,Unknown,500,375,44.9506,101.24,null,nul,G,168.00,1,no,M,no,5.0,2013-05-30 00:00:00,1900-01-01,1900-01-01
500,996,HL Bottom Bracket,BB-9108,Unknown,500,375,53.9416,121.49,null,nul,G,170.00,1,no,H,no,5.0,2013-05-30 00:00:00,1900-01-01,1900-01-01
501,997,"Road-750 Black, 44",BK-R19B-44,Black,100,75,343.6496,539.99,44,CM,LB,19.77,4,R,L,U,2.0,2013-05-30 00:00:00,1900-01-01,1900-01-01
502,998,"Road-750 Black, 48",BK-R19B-48,Black,100,75,343.6496,539.99,48,CM,LB,20.13,4,R,L,U,2.0,2013-05-30 00:00:00,1900-01-01,1900-01-01


In [26]:
#Production.Work
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        work_order_df = pd.read_sql_query("SELECT WorkOrderID, ProductID, OrderQty, StockedQty, ScrappedQty, StartDate, EndDate, DueDate FROM Production.WorkOrder", source_conn)
        work_order_routing_df = pd.read_sql_query("SELECT WorkOrderID, OperationSequence, LocationID, ScheduledStartDate, ScheduledEndDate, ActualStartDate, ActualEndDate, ActualResourceHrs, PlannedCost, ActualCost FROM Production.WorkOrderRouting", source_conn)
         ## merging dataframes
        work_order_df = pd.merge(work_order_routing_df, work_order_df, on='WorkOrderID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
        # Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Work]
        (WorkOrderID, ProductID, OperationSequence, LocationID, ScheduledStartDate, ScheduledEndDate, ActualStartDate, ActualEndDate, ActualResourceHrs, PlannedCost, ActualCost, OrderQty, StockedQty, StartDate, EndDate, DueDate)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = work_order_df[['WorkOrderID', 'ProductID', 'OperationSequence', 'LocationID', 'ScheduledStartDate', 'ScheduledEndDate', 'ActualStartDate', 'ActualEndDate', 'ActualResourceHrs', 'PlannedCost', 'ActualCost', 'OrderQty', 'StockedQty', 'StartDate', 'EndDate', 'DueDate']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [133]:
#Production.ProductionHistory
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        ProductCostHistory_df = pd.read_sql_query("SELECT ProductID, StartDate, EndDate, StandardCost FROM Production.ProductCostHistory", source_conn)
        ProductListPriceHistory_df = pd.read_sql_query("SELECT ProductID, StartDate, EndDate, ListPrice FROM Production.ProductListPriceHistory", source_conn)
        TransactionHistory_df = pd.read_sql_query("SELECT ProductID, TransactionID, ReferenceOrderID, ReferenceOrderLineID, TransactionDate, Quantity, ActualCost FROM Production.TransactionHistory", source_conn)
        
         ## merging dataframes
        ProductCostHistory_df = pd.merge(ProductCostHistory_df, ProductListPriceHistory_df, on='ProductID', how='left')
        ProductCostHistory_df = pd.merge(ProductCostHistory_df, TransactionHistory_df, on='ProductID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
    
# Convert datetime columns to datetime format
ProductCostHistory_df['StartDate_x'] = pd.to_datetime(ProductCostHistory_df['StartDate_x'])
ProductCostHistory_df['EndDate_x'] = pd.to_datetime(ProductCostHistory_df['EndDate_x'])
ProductCostHistory_df['StartDate_y'] = pd.to_datetime(ProductCostHistory_df['StartDate_y'])
ProductCostHistory_df['EndDate_y'] = pd.to_datetime(ProductCostHistory_df['EndDate_y'])
ProductCostHistory_df['TransactionDate'] = pd.to_datetime(ProductCostHistory_df['TransactionDate'])
ProductCostHistory_df['ReferenceOrderID'].fillna(0, inplace=True)
ProductCostHistory_df['ReferenceOrderLineID'].fillna(0, inplace=True)
ProductCostHistory_df['TransactionID'].fillna(0, inplace=True)
ProductCostHistory_df['Quantity'].fillna(0.0, inplace=True)
ProductCostHistory_df['ActualCost'].fillna(0.0, inplace=True)

# Handle missing values in datetime columns (if any)
# For example, fill missing values with a default datetime value
default_datetime = pd.to_datetime('2000-01-01')
ProductCostHistory_df['StartDate_x'].fillna(default_datetime, inplace=True)
ProductCostHistory_df['EndDate_x'].fillna(default_datetime, inplace=True)
ProductCostHistory_df['StartDate_y'].fillna(default_datetime, inplace=True)
ProductCostHistory_df['EndDate_y'].fillna(default_datetime, inplace=True)
ProductCostHistory_df['TransactionDate'].fillna(default_datetime, inplace=True)
ProductCostHistory_df['ReferenceOrderID'] = ProductCostHistory_df['ReferenceOrderID'].astype(int)
ProductCostHistory_df['ReferenceOrderLineID'] = ProductCostHistory_df['ReferenceOrderLineID'].astype(int)
ProductCostHistory_df['TransactionID'] = ProductCostHistory_df['TransactionID'].astype(int)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert for ProductCostHistory
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ProductionHistory]
        (ProductID, Cost_StartDate, Cost_EndDate, StandardCost, List_StartDate, List_EndDate, ListPrice, TransactionID, ReferenceOrderID, ReferenceOrderLineID, TransactionDate, Quantity, ActualCost)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = ProductCostHistory_df[['ProductID', 'StartDate_x', 'EndDate_x', 'StandardCost', 'StartDate_y', 'EndDate_y', 'ListPrice', 'TransactionID', 'ReferenceOrderID', 'ReferenceOrderLineID', 'TransactionDate', 'Quantity', 'ActualCost']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into ProductCostHistory table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into ProductCostHistory table.


,ProductID,StartDate_x,EndDate_x,StandardCost,StartDate_y,EndDate_y,ListPrice,TransactionID,ReferenceOrderID,ReferenceOrderLineID,TransactionDate,Quantity,ActualCost
0,707,2011-05-31,2012-05-29,12.0278,2011-05-31,2012-05-29,33.6442,100139,53454,1,2013-07-31,6.0,20.994
1,707,2011-05-31,2012-05-29,12.0278,2011-05-31,2012-05-29,33.6442,100285,53458,23,2013-07-31,6.0,20.994
2,707,2011-05-31,2012-05-29,12.0278,2011-05-31,2012-05-29,33.6442,100336,53459,25,2013-07-31,6.0,20.994
3,707,2011-05-31,2012-05-29,12.0278,2011-05-31,2012-05-29,33.6442,100385,53460,31,2013-07-31,10.0,20.994
4,707,2011-05-31,2012-05-29,12.0278,2011-05-31,2012-05-29,33.6442,100486,53465,2,2013-07-31,12.0,20.994
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259355,999,2013-05-30,2000-01-01,343.6496,2013-05-30,2000-01-01,539.9900,208832,72194,0,2014-05-30,1.0,0.000
259356,999,2013-05-30,2000-01-01,343.6496,2013-05-30,2000-01-01,539.9900,209047,74143,1,2014-05-30,1.0,539.990
259357,999,2013-05-30,2000-01-01,343.6496,2013-05-30,2000-01-01,539.9900,209086,72303,0,2014-05-31,3.0,0.000
259358,999,2013-05-30,2000-01-01,343.6496,2013-05-30,2000-01-01,539.9900,209277,72415,0,2014-06-01,1.0,0.000


In [28]:
#Production.ProductInventory
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        ProductInventory_df = pd.read_sql_query("SELECT ProductID, LocationID, Shelf, Bin, Quantity FROM Production.ProductInventory", source_conn)
        Location_df = pd.read_sql_query("SELECT LocationID, Name, CostRate, Availability FROM Production.Location", source_conn)
        
        ## merging dataframes
        ProductInventory_df = pd.merge(ProductInventory_df, Location_df, on='LocationID', how='left')
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ProductInventory]
        (ProductID, LocationID, Shelf, Bin, Quantity, Name, CostRate, Availability)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = ProductInventory_df[['ProductID', 'LocationID', 'Shelf', 'Bin', 'Quantity', 'Name', 'CostRate', 'Availability']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into ProductInventory table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into ProductInventory table.


In [29]:
#Purchasing.ProductVendor
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        ProductVendor_df = pd.read_sql_query("SELECT ProductID, BusinessEntityID, AverageLeadTime, StandardPrice, LastReceiptCost, LastReceiptDate, MinOrderQty, MaxOrderQty, OnOrderQty, UnitMeasureCode FROM Purchasing.ProductVendor", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

ProductVendor_df['OnOrderQty'].fillna(0, inplace=True)
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ProductVendor]
        (ProductID, BusinessEntityID, AverageLeadTime, StandardPrice, LastReceiptCost, LastReceiptDate, MinOrderQty, MaxOrderQty, OnOrderQty, UnitMeasureCode)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        data_to_insert = ProductVendor_df[['ProductID', 'BusinessEntityID', 'AverageLeadTime', 'StandardPrice', 'LastReceiptCost', 'LastReceiptDate', 'MinOrderQty', 'MaxOrderQty', 'OnOrderQty', 'UnitMeasureCode']].values.tolist()
        cursor.executemany(insert_query, data_to_insert)
        destination_conn.commit()
        print("Data inserted successfully into ProductVendor table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into ProductVendor table.


In [30]:
#ShoppingcartItem
try:
      with pyodbc.connect(source_conn_str) as source_conn:
          Shoppingcart_df = pd.read_sql_query("SELECT ShoppingCartItemID, ShoppingCartID, Quantity, ProductID, DateCreated from Sales.ShoppingCartItem", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[ShoppingCartItem]
        (ShoppingCartItemID, ShoppingCardID, Quantity, ProductID, DateCreated)
        VALUES (?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = Shoppingcart_df[['ShoppingCartItemID', 'ShoppingCartID', 'Quantity', 'ProductID', 'DateCreated']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into ShoppingCartItem table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into ShoppingCartItem table.


In [117]:
#Orders/Northwind
try:
        with pyodbc.connect(source_conn_str_northwind) as source_conn:
            Orders_df = pd.read_sql_query("SELECT OrderID, CustomerID, EmployeeID, OrderDate, RequiredDate, ShippedDate, ShipVia, Freight, ShipName, ShipAddress, ShipCity, ShipRegion, ShipPostalCode, ShipCountry  FROM Orders", source_conn)
            OrdersDetail_df = pd.read_sql_query("SELECT OrderID, ProductID, UnitPrice, Quantity, Discount FROM Order_Details", source_conn)
            
            ##mergingOrderDetails
            Orders_df = pd.merge(Orders_df, OrdersDetail_df, on='OrderID', how='left')
            
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)
    
Orders_df['OrderDate'] = Orders_df['OrderDate'].dt.strftime('%Y-%m-%d %H:%M:%S')
Orders_df['RequiredDate'] = Orders_df['RequiredDate'].dt.strftime('%Y-%m-%d %H:%M:%S')
Orders_df['ShippedDate'].fillna('1900-01-01 00:00:00', inplace=True)
sales_order_df['Freight'].fillna(0.0, inplace=True)
sales_order_df['UnitPrice'].fillna(0.0, inplace=True)
Orders_df['ShipRegion'].fillna('Default', inplace=True)
Orders_df['ShipPostalCode'].fillna('Default', inplace=True)

try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Orders]
        (OrderID, CustomerID, EmployeeID, OrderDate, RequiredDate, ShippedDate, ShipVia, Freight, ShipName, ShipAdress, ShipCity, ShipRegion, ShipPostalCode, ShipCountry, ProductID, UnitPrice, Quantity, Discount)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = Orders_df[['OrderID', 'CustomerID', 'EmployeeID', 'OrderDate', 'RequiredDate', 'ShippedDate', 'ShipVia', 'Freight', 'ShipName', 'ShipAddress', 'ShipCity', 'ShipRegion', 'ShipPostalCode', 'ShipCountry', 'ProductID', 'UnitPrice', 'Quantity', 'Discount']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into OrderDetails table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data inserted successfully into OrderDetails table.


,OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry,ProductID,UnitPrice,Quantity,Discount
0,10248,VINET,5,1996-07-04 00:00:00,1996-08-01 00:00:00,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Default,51100,France,11,14.00,12,0.00
1,10248,VINET,5,1996-07-04 00:00:00,1996-08-01 00:00:00,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Default,51100,France,42,9.80,10,0.00
2,10248,VINET,5,1996-07-04 00:00:00,1996-08-01 00:00:00,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Default,51100,France,72,34.80,5,0.00
3,10249,TOMSP,6,1996-07-05 00:00:00,1996-08-16 00:00:00,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Default,44087,Germany,14,18.60,9,0.00
4,10249,TOMSP,6,1996-07-05 00:00:00,1996-08-16 00:00:00,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,Default,44087,Germany,51,42.40,40,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150,11077,RATTC,1,1998-05-06 00:00:00,1998-06-03 00:00:00,1900-01-01,2,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,USA,64,33.25,2,0.03
2151,11077,RATTC,1,1998-05-06 00:00:00,1998-06-03 00:00:00,1900-01-01,2,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,USA,66,17.00,1,0.00
2152,11077,RATTC,1,1998-05-06 00:00:00,1998-06-03 00:00:00,1900-01-01,2,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,USA,73,15.00,2,0.01
2153,11077,RATTC,1,1998-05-06 00:00:00,1998-06-03 00:00:00,1900-01-01,2,8.53,Rattlesnake Canyon Grocery,2817 Milton Dr.,Albuquerque,NM,87110,USA,75,7.75,4,0.00


In [32]:
#unitmeasure
# Establish connection to the source SQL Server database and read
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        unitmeasure_df = pd.read_sql_query("SELECT UnitMeasureCode, Name FROM Production.UnitMeasure", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()

        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[UnitMeasure]
        (UnitMeasureCode, Name)
        VALUES (?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = unitmeasure_df[['UnitMeasureCode', 'Name']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert) 
        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [33]:
#BillOfMaterials
# Fetch data from source
try:
    with pyodbc.connect(source_conn_str) as source_conn:
        bill_of_material_df = pd.read_sql_query("""SELECT BillOfMaterialsID, ProductAssemblyID, ComponentID, StartDate, EndDate, UnitMeasureCode, BOMlevel, PerAssemblyQty FROM Production.BillOfMaterials""", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Validate and clean date columns
date_columns = ['StartDate', 'EndDate']
for column in date_columns:
    bill_of_material_df[column] = pd.to_datetime(bill_of_material_df[column], errors='coerce')

# Drop rows with invalid StartDate
bill_of_material_df.dropna(subset=['StartDate'], inplace=True)

# Replace NaT in EndDate with None
bill_of_material_df['EndDate'] = bill_of_material_df['EndDate'].apply(lambda x: None if pd.isna(x) else x)

# Convert datetime columns to strings in 'YYYY-MM-DD' format, or None
bill_of_material_df['StartDate'] = bill_of_material_df['StartDate'].apply(lambda x: x.strftime('%Y-%m-%d') if not pd.isna(x) else None)
bill_of_material_df['EndDate'] = bill_of_material_df['EndDate'].apply(lambda x: x.strftime('%Y-%m-%d') if not pd.isna(x) else None)

# Ensure PerAssemblyQty is of type float
bill_of_material_df['PerAssemblyQty'] = bill_of_material_df['PerAssemblyQty'].astype(float)

# Handle missing and infinite values in ProductAssemblyID
bill_of_material_df['ProductAssemblyID'] = bill_of_material_df['ProductAssemblyID'].fillna(0).astype(int)

# Print data types of each column
print("Data types of DataFrame columns:")
print(bill_of_material_df.dtypes)

# Insert data into destination
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[BillOfMaterial]
        (BillOfMaterialsID, ProductAssemblyID, ComponentID, StartDate, EndDate, UnitMeasureCode, BOMlevel, PerAssemblyQty)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = bill_of_material_df[
            ['BillOfMaterialsID', 'ProductAssemblyID', 'ComponentID', 'StartDate', 'EndDate', 'UnitMeasureCode', 'BOMlevel', 'PerAssemblyQty']
        ].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert)

        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully into BillOfMaterial table.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server:", e)

Data types of DataFrame columns:
BillOfMaterialsID      int64
ProductAssemblyID      int32
ComponentID            int64
StartDate             object
EndDate               object
UnitMeasureCode       object
BOMlevel               int64
PerAssemblyQty       float64
dtype: object
Data inserted successfully into BillOfMaterial table.


In [34]:
#Product/Northwind
try:
    with pyodbc.connect(source_conn_str_northwind) as source_conn:
        NorthWindProducts_df = pd.read_sql_query("SELECT * FROM dbo.Products", source_conn)
        NorthWindCategories_df = pd.read_sql_query("SELECT * FROM dbo.Categories", source_conn)
        #merge on CategoryID
        NorthWindSuppliers_df = pd.read_sql_query("SELECT SupplierID, CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone, Fax FROM dbo.Suppliers", source_conn)  
        #merge on SupplierID
        NorthWindProducts_df = pd.merge(NorthWindProducts_df, NorthWindCategories_df, on='CategoryID', how='left')
        NorthWindProducts_df = pd.merge(NorthWindProducts_df, NorthWindSuppliers_df, on='SupplierID', how='left')

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Product]
        (ProductID, ProductName, SupplierID, CategoryID, QuantityPerUnit, UnitPrice, UnitsInStock, UnitsOnOrder, ReorderLevel, Discontinued, CategoryName, Description, Picture, CompanyName, ContactName, ContactTitle, Adress, City, Region, PostalCode, Country, Phone, Fax)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = NorthWindProducts_df[['ProductID', 'ProductName', 'SupplierID', 'CategoryID', 'QuantityPerUnit', 'UnitPrice', 'UnitsInStock', 'UnitsOnOrder', 'ReorderLevel', 'Discontinued', 'CategoryName', 'Description', 'Picture', 'CompanyName', 'ContactName', 'ContactTitle', 'Address', 'City', 'Region', 'PostalCode', 'Country', 'Phone', 'Fax']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert) 
        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [35]:
#Region/Northwind
try:
    with pyodbc.connect(source_conn_str_northwind) as source_conn:
        NorthWindRegion_df = pd.read_sql_query("SELECT RegionID, RegionDescription FROM dbo.Region", source_conn)
        NorthWindTerritories_df = pd.read_sql_query("SELECT TerritoryID, TerritoryDescription, RegionID FROM dbo.Territories", source_conn)
        #merge on SupplierID
        NorthWindRegion_df = pd.merge(NorthWindRegion_df, NorthWindTerritories_df, on='RegionID', how='left')

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Region]
        (RegionID, RegionDescription, TerritoryID, TerritoryDescription)
        VALUES (?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = NorthWindRegion_df[['RegionID', 'RegionDescription', 'TerritoryID', 'TerritoryDescription']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert) 
        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [36]:
#EmployeeTerritories/Northwind
try:
    with pyodbc.connect(source_conn_str_northwind) as source_conn:
        NorthWindEmployeeTerritories_df = pd.read_sql_query("SELECT EmployeeID, TerritoryID FROM dbo.EmployeeTerritories", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[EmployeeTerritories]
        (EmployeeID, TerritoryID)
        VALUES (?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = NorthWindEmployeeTerritories_df[['EmployeeID', 'TerritoryID']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert) 
        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [37]:
#Employees/Northwind
try:
    with pyodbc.connect(source_conn_str_northwind) as source_conn:
        NorthWindEmployees_df = pd.read_sql_query("SELECT * FROM dbo.Employees", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

NorthWindEmployees_df['EmployeeID'] = NorthWindEmployees_df['EmployeeID'].astype(int)
NorthWindEmployees_df['ReportsTo'] = NorthWindEmployees_df['ReportsTo'].fillna(0).astype(int)
# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Employees]
        (EmployeeID, LastName, FirstName, Title, TitleOfCourtesy, BirthDate, HireDate, Adress, City, Region, PostalCode, Country, HomePhone, Extension, Photo, Notes, ReportsTo, PhotoPath)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = NorthWindEmployees_df[['EmployeeID', 'LastName', 'FirstName', 'Title', 'TitleOfCourtesy', 'BirthDate', 'HireDate', 'Address', 'City', 'Region', 'PostalCode', 'Country', 'HomePhone', 'Extension', 'Photo', 'Notes', 'ReportsTo', 'PhotoPath']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert) 
        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [38]:
#Shippers/Northwind
try:
    with pyodbc.connect(source_conn_str_northwind) as source_conn:
        NorthWindShippers_df = pd.read_sql_query("SELECT * FROM dbo.Shippers", source_conn)

except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Shippers]
        (ShipperID, CompanyName, Phone)
        VALUES (?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = NorthWindShippers_df[['ShipperID', 'CompanyName', 'Phone']].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert) 
        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e:
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


In [87]:
#Customer/Northwind
try:
    with pyodbc.connect(source_conn_str_northwind) as source_conn:
        NorthWindCustomer_df = pd.read_sql_query("SELECT * FROM dbo.Customers", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

try:
    with pyodbc.connect(source_conn_str) as source_conn:
        Customers_df = pd.read_sql_query("SELECT * FROM Sales.Customer", source_conn)
except pyodbc.Error as e:
    print("Error while connecting to the source SQL Server:", e)

# Convert 'CustomerID' to string type
NorthWindCustomer_df['CustomerID'] = NorthWindCustomer_df['CustomerID'].astype(str)
Customers_df['CustomerID'] = Customers_df['CustomerID'].astype(str)

# Set 'CustomerID' as the index for both DataFrames
NorthWindCustomer_df.set_index('CustomerID', inplace=True)
Customers_df.set_index('CustomerID', inplace=True)

# Concatenate the two DataFrames along the index
merged_customers_df = pd.concat([NorthWindCustomer_df, Customers_df], join='outer')

# Reset the index to convert 'CustomerID' back to a column
merged_customers_df.reset_index(inplace=True)

merged_customers_df['PersonID'].fillna(0, inplace=True)
merged_customers_df['StoreID'].fillna(0, inplace=True)
merged_customers_df['TerritoryID'].fillna(0, inplace=True)
merged_customers_df['AccountNumber'].fillna('Default', inplace=True)
merged_customers_df['Region'].fillna('Default', inplace=True)
merged_customers_df['Country'].fillna('Default', inplace=True)
merged_customers_df['Address'].fillna('Default', inplace=True)
merged_customers_df['CustomerID'].fillna('Default', inplace=True)
merged_customers_df['CompanyName'].fillna('Default', inplace=True)
merged_customers_df['ContactName'].fillna('Default', inplace=True)
merged_customers_df['ContactTitle'].fillna('Default', inplace=True)
merged_customers_df['City'].fillna('Default', inplace=True)
merged_customers_df['PostalCode'].fillna('Default', inplace=True)
merged_customers_df['Phone'].fillna('Default', inplace=True)
merged_customers_df['Fax'].fillna('Default', inplace=True)

merged_customers_df['PersonID'] = merged_customers_df['PersonID'].astype(int)
merged_customers_df['StoreID'] = merged_customers_df['StoreID'].astype(int)
merged_customers_df['TerritoryID'] = merged_customers_df['TerritoryID'].astype(int)
merged_customers_df['AccountNumber'] = merged_customers_df['AccountNumber'].astype(str)
merged_customers_df['Address'] = merged_customers_df['Address'].astype(str)

# Establish connection to the destination SQL Server database and insert data
try:
    with pyodbc.connect(destination_conn_str) as destination_conn:
        cursor = destination_conn.cursor()
        # Batch insert
        insert_query = """
        INSERT INTO [DWUnitedOutdoors].[dbo].[Customer]
        (CustomerID, PersonID, StoreID, TerritoryID, AccountNumber, CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone, Fax)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        # Convert DataFrame to a list of tuples for batch insertion
        data_to_insert = merged_customers_df[[
            'CustomerID', 'PersonID', 'StoreID', 'TerritoryID', 'AccountNumber',
            'CompanyName', 'ContactName', 'ContactTitle', 'Address', 'City', 'Region',
            'PostalCode', 'Country', 'Phone', 'Fax'
        ]].values.tolist()

        # Executing batch insert
        cursor.executemany(insert_query, data_to_insert) 
        # Commit transaction
        destination_conn.commit()

        print("Data inserted successfully.")
except pyodbc.Error as e: 
    print("Error while connecting to the destination SQL Server or inserting data:", e)

Data inserted successfully.


,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,Default,12209,Germany,030-0074321,030-0076545,0,0,0,Default,NaN,NaT
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,Default,05021,Mexico,(5) 555-4729,(5) 555-3745,0,0,0,Default,NaN,NaT
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,Default,05023,Mexico,(5) 555-3932,Default,0,0,0,Default,NaN,NaT
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,Default,WA1 1DP,UK,(171) 555-7788,(171) 555-6750,0,0,0,Default,NaN,NaT
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,Default,S-958 22,Sweden,0921-12 34 65,0921-12 34 67,0,0,0,Default,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19906,30114,Default,Default,Default,Default,Default,Default,Default,Default,Default,Default,1985,1986,7,AW00030114,97154F3D-28F1-4B15-AE03-9518B781ECE3,2014-09-12 11:15:07.263
19907,30115,Default,Default,Default,Default,Default,Default,Default,Default,Default,Default,1987,1988,6,AW00030115,E4CF8FD5-30A4-4B8E-8FD8-47032E255778,2014-09-12 11:15:07.263
19908,30116,Default,Default,Default,Default,Default,Default,Default,Default,Default,Default,1989,1990,4,AW00030116,EC409609-D25D-41B8-9D15-A1AA6E89FC77,2014-09-12 11:15:07.263
19909,30117,Default,Default,Default,Default,Default,Default,Default,Default,Default,Default,1991,1992,4,AW00030117,6F08E2FB-1CD3-4F6E-A2E6-385669598B19,2014-09-12 11:15:07.263
